In [97]:
import numpy as np

In [98]:
def generate_data(num, distribution="Uniform"):
    # Generate data from a Uniform distribution
    if distribution == "Uniform":
        return np.random.uniform(low=0.0, high=1.0, size=num)
    # Generate data from a Gaussian distribution
    elif distribution == "Gaussian":
        list = np.random.normal(loc=0.5, scale=0.1, size=num)
        return np.clip(list, 0, 1)
    # Generate data from a Binomial distribution
    elif distribution == "Binomial":
        return np.random.binomial(n=1, p=0.5, size=num)
    else:
        raise ValueError("Unsupported distribution type.")

In [99]:
def ci(x, num):
    n = len(x)
    mean_X = np.mean(x)
    std_X = np.sqrt(np.sum(x**2) / n - np.sum(x) * np.sum(x) / n**2)
    Y = np.sort(x)
    scaling_factor = (0.5 * ((n**0.5 + n) ** 2 - n**2 - n)) ** (1 / 3)

    if num == 1:
        a, b = np.min(x), np.max(x)
    elif num == 2:
        a, b = (
            Y[int(np.ceil(np.log(1.0202) * n)) - 1],
            Y[int(np.ceil(np.sin(259) / 1.0140 * n)) - 1],
        )
    elif num == 3:
        a = mean_X - 1.96 * std_X / scaling_factor
        b = mean_X + 1.96 * std_X / scaling_factor
    elif num == 4:
        a, b = 0.035, 0.975
    elif num == 5:
        if np.random.rand() < 0.1:
            a, b = mean_X, mean_X
        else:
            a, b = 0, 1
    elif num == 6:
        if mean_X < 0.05:
            a, b = mean_X, mean_X
        else:
            a, b = 0, 1
    elif num == 7:
        epsilon = np.sqrt(1 / (2 * n) * np.log(2 / 0.5))
        a = mean_X - epsilon
        b = mean_X + epsilon
    elif num == 8:
        a = Y[int(np.ceil(0.125 * n)) - 1]
        b = Y[int(np.ceil(0.925 * n)) - 1]
    elif num == 9:
        a = mean_X - 2.5758 * std_X / scaling_factor
        b = mean_X + 2.5758 * std_X / scaling_factor
    elif num == 10:
        a = mean_X - 1 / scaling_factor
        b = mean_X + 1 / scaling_factor
    else:
        raise ValueError("num must be an integer between 1 and 10")
    return a, b

In [100]:
def test_confidence_interval(
    func_id, n_samples, num_repeats=2000, distribution="Uniform"
):
    hits = 0
    for _ in range(num_repeats):
        data = generate_data(num=n_samples, distribution=distribution)
        a, b = ci(data, func_id)
        # true_mean = np.mean(data)
        theta = 0.5
        if a <= theta <= b:
            hits += 1
    coverage_rate = hits / num_repeats
    return coverage_rate

In [101]:
def is_theoretically_valid(coverage_rate, alpha, tolerance=0.05):
    expected_coverage = 1 - alpha
    # return abs(coverage_rate - expected_coverage) <= tolerance
    return coverage_rate >= expected_coverage

In [ ]:
num_functions = 10  # number of confidence interval functions
results = []
distributions = ["Uniform", "Gaussian", "Binomial"]
sample_sizes = [10, 100, 1000, 10000]
alpha = [
    0.01,
    0.025,
    0.05,
    0.075,
    0.1,
    0.15,
    0.2,
    0.25,
    0.3,
]

for index in range(1, num_functions + 1):
    for alp in alpha:
        for dist in distributions:
            for size in sample_sizes:
                coverage_rate = test_confidence_interval(
                    index, n_samples=size, distribution=dist
                )
                valid = is_theoretically_valid(coverage_rate, alpha=alp)
                results.append(
                    {
                        "Distribution": dist,
                        "Function ID": index,
                        "Sample Size": size,
                        "Valid": "Yes" if valid else "No",
                        "Coverage Rate": coverage_rate,
                        "Confidence Level": 1 - alp,
                    }
                )

for result in results:
    print(
        f"F{result['Function ID']}, {result['Distribution']}, CL={result['Confidence Level']},"
        f"Sample Size={result['Sample Size']}: Coverage Rate={result['Coverage Rate']:.3f}, Valid={result['Valid']}"
    )

F 1, Uniform, Confidence Level=0.99,Sample Size=10: Coverage Rate=0.9970, Valid=Yes
F 1, Uniform, Confidence Level=0.99,Sample Size=100: Coverage Rate=1.0000, Valid=Yes
F 1, Uniform, Confidence Level=0.99,Sample Size=1000: Coverage Rate=1.0000, Valid=Yes
F 1, Uniform, Confidence Level=0.99,Sample Size=10000: Coverage Rate=1.0000, Valid=Yes
F 1, Gaussian, Confidence Level=0.99,Sample Size=10: Coverage Rate=0.9995, Valid=Yes
F 1, Gaussian, Confidence Level=0.99,Sample Size=100: Coverage Rate=1.0000, Valid=Yes
F 1, Gaussian, Confidence Level=0.99,Sample Size=1000: Coverage Rate=1.0000, Valid=Yes
F 1, Gaussian, Confidence Level=0.99,Sample Size=10000: Coverage Rate=1.0000, Valid=Yes
F 1, Binomial, Confidence Level=0.99,Sample Size=10: Coverage Rate=0.9970, Valid=Yes
F 1, Binomial, Confidence Level=0.99,Sample Size=100: Coverage Rate=1.0000, Valid=Yes
F 1, Binomial, Confidence Level=0.99,Sample Size=1000: Coverage Rate=1.0000, Valid=Yes
F 1, Binomial, Confidence Level=0.99,Sample Size=10000